In [ ]:
import pandas as pd
import numpy as np

def load_data():
    """데이터 로드 함수"""
    examinees_df = pd.read_csv('./datas/수험자_1000.csv')
    questions_df = pd.read_csv('./datas/문항_50000_Questions_Data.csv')
    exam_df = pd.read_csv('exam1.csv')
    
    # 문항 데이터 병합
    questions_cols = ['문제번호', '문항코드', '평가종류', '배점', '출제단계', '문항유형', '난이도', '평가영역', '평가항목']
    items_cols = ['문항코드', '감성', '정답']
    questions_df = pd.merge(exam_df[questions_cols], questions_df[items_cols], on='문항코드')
    
    # 문항유형 결측치 및 데이터 타입 처리
    questions_df['문항유형'] = questions_df['문항유형'].fillna('객관식').astype(str)
    
    return examinees_df, questions_df

def generate_score(korean_level, pass_rate=0.6):
    """점수 및 정답률 생성 함수"""
    # 평균 점수를 pass_rate에 따라 조정
    mean_score = 70 + (pass_rate - 0.5) * 60  # 기본 평균 70에서 pass_rate에 따라 조정
    score = np.clip(np.random.normal(mean_score, 20), 0, 100)
    
    base_correct_rate = score / 100
    correct_rate = np.clip(base_correct_rate * (korean_level / 5), 0, 1)
    
    return score, correct_rate if 0 < correct_rate < 1 else 0.5

def generate_answers(questions, ox_list):
    """답안 생성 함수"""
    selected_answers = []
    all_options = set(range(1, 5))  # 모든 가능한 답안 옵션

    for q_code, q_type, ox in zip(questions['문항코드'], questions['문항유형'], ox_list):
        if q_type == '객관식':
            correct_answer = questions.loc[questions['문항코드'] == q_code, '정답'].iloc[0]
            if ox == 'O':
                answer = correct_answer
            else:
                wrong_answers = list(all_options - {correct_answer})
                answer = np.random.choice(wrong_answers)
        else:
            answer = 0  # 객관식이 아닌 경우

        selected_answers.append(answer)

    return selected_answers

def generate_response(examinee, questions, pass_rate=0.6):
    """응답 데이터 생성 함수"""
    num_questions = len(questions)
    korean_level = examinee['한국어 수준'] if 1 <= examinee['한국어 수준'] <= 5 else 3
    
    score, correct_rate = generate_score(korean_level, pass_rate)
    ox_list = np.random.choice(['O', 'X'], size=num_questions, p=[correct_rate, 1-correct_rate])
    
    # score는 ox_list를 기준으로 실제 문제에 배정된 배점의 총합으로 계산
    correct_questions = questions[ox_list == 'O']
    score = correct_questions['배점'].sum()
    
    selected_answers = generate_answers(questions, ox_list)
    
    return {
        '외국인 등록번호': examinee['외국인 등록번호'],
        '이름': examinee['이름'],
        '성별': examinee['성별'],
        '나이': examinee['나이'],
        '국적': examinee['국적'],
        '거주지': examinee['거주지'],
        '직업': examinee['직업'],
        '한국어 수준': korean_level,
        '점수': round(score, 2),
        '결과': 'PASS' if score >= 60 else 'FAIL',
        'OX리스트': ''.join(ox_list),
        '선택답안': ','.join(map(str, selected_answers)),
        '문항코드리스트': ','.join(map(str, questions['문항코드'].tolist()))
    }

def main():
    """메인 함수"""
    examinees_df, questions_df = load_data()
    
    # 30명의 수험자 랜덤 추출
    selected_examinees = examinees_df.sample(n=30, random_state=42)
    
    # 30명의 응답 데이터 생성
    responses = []
    pass_rate = 0.6  # 원하는 PASS율 설정
    for _, examinee in selected_examinees.iterrows():
        try:
            response = generate_response(examinee, questions_df, pass_rate)
            responses.append(response)
        except Exception as e:
            print(f"수험자 {examinee['이름']}의 응답 생성 중 오류 발생: {e}")
    
    # 응답이 생성되지 않은 수험자가 있는지 확인
    if len(responses) < 30:
        print(f"{30 - len(responses)}명의 응답이 생성되지 않았습니다.")
    
    # DataFrame으로 변환
    response_df = pd.DataFrame(responses)
    
    # 결과 출력 (첫 5개 행만)
    print(response_df.head())
    
    # CSV 파일로 저장
    response_df.to_csv('응답_데이터.csv', index=False, encoding='utf-8-sig')
    print("응답 데이터가 '응답_데이터.csv' 파일로 저장되었습니다.")

if __name__ == "__main__":
    main()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import matplotlib.font_manager as fm

# 한글 폰트 설정
# os를 체크하여 맥과 그외를 구분하여 폰트 경로 설정

if platform.system() == 'Darwin':
    font_path = '/Users/ldm/Library/Fonts/NanumGothic.ttf'
else:
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'


font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)


# Add font
fm.fontManager.addfont(font_path)

# Set as default font
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import platform

# 데이터 로드
df = pd.read_csv('응답_데이터.csv')

# 데이터 확인
print(df.head())
print(df.info())

# 1. 점수 분포 히스토그램
plt.figure(figsize=(10, 6))
sns.histplot(df['점수'], kde=True)
plt.title('점수 분포 히스토그램')
plt.xlabel('점수')
plt.ylabel('빈도')
plt.show()

# 2. 통과율 원형 차트
plt.figure(figsize=(8, 8))
pass_counts = df['결과'].value_counts()
plt.pie(pass_counts.values, labels=pass_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('결과 분포 원형 차트')
plt.show()

# 3. 한국어 수준별 평균 점수 막대 그래프
plt.figure(figsize=(10, 6))
sns.barplot(x='한국어 수준', y='점수', data=df)
plt.title('한국어 수준별 평균 점수')
plt.xlabel('한국어 수준')
plt.ylabel('평균 점수')
plt.show()

# 4. 문항별 정답률 선 그래프
ox_data = df['OX리스트'].apply(lambda x: pd.Series(list(x)))
correct_rates = (ox_data == 'O').mean()
plt.figure(figsize=(12, 6))
plt.plot(correct_rates.index, correct_rates.values, marker='o')
plt.title('문항별 정답률')
plt.xlabel('문항 번호')
plt.ylabel('정답률')
plt.show()

# 5. 국적별 평균 점수 박스 플롯
plt.figure(figsize=(12, 6))
sns.boxplot(x='국적', y='점수', data=df)
plt.title('국적별 점수 분포')
plt.xlabel('국적')
plt.ylabel('점수')
plt.xticks(rotation=45)
plt.show()

# 6. 나이와 점수의 산점도
plt.figure(figsize=(10, 6))
sns.scatterplot(x='나이', y='점수', data=df)
plt.title('나이와 점수의 관계')
plt.xlabel('나이')
plt.ylabel('점수')
plt.show()

# 7. 성별에 따른 점수 비교 바이올린 플롯
plt.figure(figsize=(10, 6))
sns.violinplot(x='성별', y='점수', data=df)
plt.title('성별에 따른 점수 분포')
plt.xlabel('성별')
plt.ylabel('점수')
plt.show()

# 8. 직업별 평균 점수 막대 그래프
plt.figure(figsize=(12, 6))
sns.barplot(x='직업', y='점수', data=df)
plt.title('직업별 평균 점수')
plt.xlabel('직업')
plt.ylabel('평균 점수')
plt.xticks(rotation=45)
plt.show()

# 9. 거주지별 통과율 막대 그래프
pass_rate = df.groupby('거주지')['결과'].apply(lambda x: (x == 'PASS').mean())
plt.figure(figsize=(12, 6))
pass_rate.plot(kind='bar')
plt.title('거주지별 통과율')
plt.xlabel('거주지')
plt.ylabel('통과율')
plt.xticks(rotation=45)
plt.show()

# 10. 문항 유형별 정답률 (데이터가 없으므로 스킵)

# 11. 한국어 수준과 통과율의 관계 (수정된 버전)
plt.figure(figsize=(10, 6))
pass_rate_by_level = df.groupby('한국어 수준')['결과'].apply(lambda x: (x == 'PASS').mean())
pass_rate_by_level.plot(kind='bar')
plt.title('한국어 수준과 통과율의 관계')
plt.xlabel('한국어 수준')
plt.ylabel('통과율')
plt.xticks(rotation=45)
plt.show()

# 12. 점수와 OX 패턴의 상관관계 매트릭스
ox_data = df['OX리스트'].apply(lambda x: pd.Series(list(x)))
ox_data = ox_data.replace({'O': 1, 'X': 0})
correlation_matrix = pd.concat([df['점수'], ox_data], axis=1).corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('점수와 OX 패턴의 상관관계')
plt.show()

print("모든 시각화가 완료되었습니다.")

In [12]:
import pandas as pd
import numpy as np

def load_data():
    """데이터 로드 함수"""
    examinees_df = pd.read_csv('./datas/수험자_1000.csv')
    questions_df = pd.read_csv('./datas/문항_50000_Questions_Data.csv')
    exam_df = pd.read_csv('exam1.csv')
    
    # 문항 데이터 병합
    questions_cols = ['문제번호', '문항코드', '평가종류', '배점', '출제단계', '문항유형', '난이도', '평가영역', '평가항목']
    items_cols = ['문항코드', '감성', '정답']
    questions_df = pd.merge(exam_df[questions_cols], questions_df[items_cols], on='문항코드')
    
    # 문항유형 결측치 및 데이터 타입 처리
    questions_df['문항유형'] = questions_df['문항유형'].fillna('객관식').astype(str)
    
    return examinees_df, questions_df

def generate_score(korean_level):
    """한국어 수준을 고려하여 점수를 랜덤하게 생성하는 함수"""
    # 한국어 수준에 따른 평균 점수 설정
    mean_scores = {1: 40, 2: 50, 3: 60, 4: 70, 5: 80}
    mean_score = mean_scores.get(korean_level, 60)
    score = np.clip(np.random.normal(mean_score, 10), 0, 100)
    return score

def generate_answers(questions, ox_list):
    """답안 생성 함수"""
    selected_answers = []
    all_options = set(range(1, 5))  # 모든 가능한 답안 옵션

    for q_code, q_type, ox in zip(questions['문항코드'], questions['문항유형'], ox_list):
        if q_type == '객관식':
            correct_answer = questions.loc[questions['문항코드'] == q_code, '정답'].iloc[0]
            if ox == 'O':
                answer = correct_answer
            else:
                wrong_answers = list(all_options - {correct_answer})
                answer = np.random.choice(wrong_answers)
        else:
            answer = 0  # 객관식이 아닌 경우

        selected_answers.append(answer)

    return selected_answers

def generate_response(examinee, questions):
    """응답 데이터 생성 함수"""
    num_questions = len(questions)
    korean_level = examinee['한국어 수준'] if 1 <= examinee['한국어 수준'] <= 5 else 3

    # 점수 랜덤 생성
    score = generate_score(korean_level)

    total_possible_score = questions['배점'].sum()
    # 필요한 정답 개수 계산
    num_correct_answers = int((score / total_possible_score) * num_questions)

    # 정답 개수만큼 랜덤하게 문항 선택하여 'O', 나머지는 'X'로 설정
    ox_list = np.array(['X'] * num_questions)
    correct_indices = np.random.choice(num_questions, num_correct_answers, replace=False)
    ox_list[correct_indices] = 'O'

    # 실제 점수 계산 (배점을 고려)
    correct_questions = questions.iloc[correct_indices]
    actual_score = correct_questions['배점'].sum()

    # 결과 결정
    result = 'PASS' if actual_score >= 60 else 'FAIL'

    selected_answers = generate_answers(questions, ox_list)

    return {
        '외국인 등록번호': examinee['외국인 등록번호'],
        '이름': examinee['이름'],
        '성별': examinee['성별'],
        '나이': examinee['나이'],
        '국적': examinee['국적'],
        '거주지': examinee['거주지'],
        '직업': examinee['직업'],
        '한국어 수준': korean_level,
        '점수': round(actual_score, 2),
        '결과': result,
        'OX리스트': ''.join(ox_list),
        '선택답안': ','.join(map(str, selected_answers)),
        '문항코드리스트': ','.join(map(str, questions['문항코드'].tolist()))
    }

def adjust_pass_rate(responses, desired_pass_rate):
    """PASS율을 조정하는 함수"""
    df = pd.DataFrame(responses)
    current_pass_rate = df['결과'].value_counts(normalize=True).get('PASS', 0)

    if current_pass_rate == desired_pass_rate:
        return responses  # PASS율이 원하는 값이면 그대로 반환

    # PASS율을 맞추기 위해 FAIL 수험자를 PASS로 또는 그 반대로 조정
    pass_gap = desired_pass_rate - current_pass_rate
    num_examinees = len(df)
    num_to_adjust = int(abs(pass_gap) * num_examinees)

    if pass_gap > 0:
        # 더 많은 수험자가 PASS해야 함 -> FAIL 수험자 중 일부를 PASS로 조정
        fail_indices = df[df['결과'] == 'FAIL'].index
        indices_to_adjust = np.random.choice(fail_indices, num_to_adjust, replace=False)
    else:
        # PASS 수험자 중 일부를 FAIL로 조정
        pass_indices = df[df['결과'] == 'PASS'].index
        indices_to_adjust = np.random.choice(pass_indices, num_to_adjust, replace=False)

    for idx in indices_to_adjust:
        response = responses[idx]
        # 점수를 약간 조정하여 결과를 반대로 변경
        if pass_gap > 0:
            # FAIL -> PASS
            additional_score = 60 - response['점수'] + np.random.uniform(0.1, 5)
            response['점수'] += additional_score
            response['결과'] = 'PASS'
        else:
            # PASS -> FAIL
            reduction = response['점수'] - 60 + np.random.uniform(0.1, 5)
            response['점수'] -= reduction
            response['결과'] = 'FAIL'
        responses[idx] = response

    return responses

def main():
    """메인 함수"""
    examinees_df, questions_df = load_data()

    # 30명의 수험자 랜덤 추출
    selected_examinees = examinees_df.sample(n=30, random_state=42).reset_index(drop=True)

    # 30명의 응답 데이터 생성
    responses = []
    for idx, examinee in selected_examinees.iterrows():
        try:
            response = generate_response(examinee, questions_df)
            responses.append(response)
        except Exception as e:
            print(f"수험자 {examinee['이름']}의 응답 생성 중 오류 발생: {e}")

    # 원하는 PASS율 설정
    desired_pass_rate = 0.6  # 원하는 PASS율 설정 (0.0 ~ 1.0)

    # PASS율 조정
    responses = adjust_pass_rate(responses, desired_pass_rate)

    # DataFrame으로 변환
    response_df = pd.DataFrame(responses)

    # 결과 출력 (첫 5개 행만)
    print(response_df.head())

    # PASS율 확인
    actual_pass_rate = response_df['결과'].value_counts(normalize=True).get('PASS', 0)
    print(f"실제 PASS율: {actual_pass_rate * 100:.2f}%")

    # CSV 파일로 저장
    response_df.to_csv('응답_데이터.csv', index=False, encoding='utf-8-sig')
    print("응답 데이터가 '응답_데이터.csv' 파일로 저장되었습니다.")

if __name__ == "__main__":
    main()


     외국인 등록번호         이름 성별  나이      국적 거주지    직업  한국어 수준         점수    결과  \
0  3490941780    Gulnara  남  36  우즈베키스탄  울산   노동자       2  60.000000  PASS   
1  4870107052    Dilnoza  여  52  우즈베키스탄  광주   예술가       3  70.000000  PASS   
2  8426465635        Ali  남  18  우즈베키스탄  인천   예술가       1  64.717796  PASS   
3  7784311902  Narantuya  남  37      몽골  울산   예술가       2  61.730946  PASS   
4  5452114233    Preecha  남  18      태국  대구  언어연수       4  62.000000  PASS   

                                               OX리스트  \
0  XOOXOOXXOXOXOXXOOXOOOXOOXXXOXOOOOOOXOOXOOXOXXX...   
1  OXOOOOOOXOOXXXOXOXOXOOOOOOXXXOOOOOOXOXOOOXOOOO...   
2  XXXOXXXOXOOXXOOXXOXXXXXOOOXOOXOXXXXXXXOOXOOXOX...   
3  XOOOOOXOOOOOOXXXXXXXXOXOOOXOXOXOOXOOOXXOOOXOXX...   
4  OOOXXXOOOOOOXOOOXOXXOXXOOXXXXOOOXXOOOOOOOXXOOX...   

                                                선택답안  \
0  1,4,2,2,1,3,3,3,2,1,2,3,2,2,3,3,2,4,2,3,2,4,3,...   
1  4,2,2,3,1,3,4,4,4,4,2,1,3,3,4,4,2,3,2,4,2,3,3,...   
2  1,3,3,3,4,4,3,4,1,4,2,2